In [1]:
!pip install wptools
!pip install wikipedia
!pip install wordcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 24.1 MB/s eta 0:00:00
  Created wheel for html2text: filename=html2text-2024.2.26-py3-none-any.whl size=33111 sha256=54ae15560d351bade40cb2cef0bc8306601215c5bdc69356101a57599826b300
  Stored in directory: /root/.cache/pip/wheels/f3/96/6d/a7eba8f80d31cbd188a2787b81514d82fc5ae6943c44777659
Successfully built html2text
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=4016da119fc13e13285c6b2d67ee4191cda95ed64c60c361274cc9fc26c1fc8d
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [2]:
import wptools
import pandas as pd

In [3]:
# Check if existing si person

In [4]:
# searching in wikipedia function
def get_wikipedia_data(person):
  try:
    page = wptools.page(person).get_parse()

    summary = page.data.get('extext', page.data.get('wikitext', 'Summary not available'))
    url= page.data.get('url', f'https://en.wikipedia.org/wiki/{person.replace(" ", "_")}')

    return{
        'name': person,
        'summary': summary[:5000]+'...',
        'url': url
    }

  except Exception as e:
      print(f'Error fetching dat for {person}: {e}')
      return None

#input name function
def get_input_names():
  input_names = input("Emnter names seperated by comma: ").split(',')
  return [name.strip() for name in input_names]

#main
people = get_input_names()
people_data = []
for person in people:
  data = get_wikipedia_data(person)
  if data:
    people_data.append(data)

Emnter names seperated by comma: Son Goku


en.wikipedia.org (parse) Son Goku
Son Goku (en) data
{
  pageid: 12763582
  parsetree: <str(962)> <root><template><title>distinguish</title>...
  requests: <list(1)> parse
  title: Son Goku
  wikibase: Q449824
  wikidata_url: https://www.wikidata.org/wiki/Q449824
  wikitext: <str(804)> {{distinguish|Sengoku (disambiguation)}}'''...
}


In [5]:
df_person = pd.DataFrame(people_data)

# Save the DataFrame to a JSON file for later use
df_person.to_json('crypto_people.json', orient='records', indent=4)

df_person

,name,summary,url
0,Son Goku,{{distinguish|Sengoku (disambiguation)}}\n'''S...,https://en.wikipedia.org/wiki/Son_Goku


In [6]:
import wptools
import wikipedia
import re
from collections import Counter
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import pandas as pd

nltk.download('stopwords')
nltk.download('punkt')  # Download the sentence tokenizer data
STOPWORDS = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
# Function to fetch info in wiki

def fetch_wikiperdia_text(name):
  try:
    search_result = wikipedia.search(name)
    if len(search_result) == 0:
      print(f'No info found for {name}')
      return None

    correct_title = search_result[0]
    page = wptools.page(correct_title).get_parse()

    if 'wikitext' in page.data:
      wikitext = page.data['wikitext']
      return wikitext
    else:
      print(f'No full text available for {name}')
      return None
  except Exception as e:
    print(f'Error in fetching data for {name}: {e}')
    return None

In [8]:
#cleaning function
def clean_sentence(sentence):
    # Remove wiki links of the form [[text]] or [[text|displayed_text]]
    cleaned = re.sub(r'\[\[([^|\]]+)(?:\|[^\]]+)?\]\]', r'\1', sentence)

    # Remove templates like {{Citation needed}} or {{Some template}} and infobox-like data
    cleaned = re.sub(r'\{\{[^}]+\}\}', '', cleaned)

    # Remove HTML-style comments <!-- comment -->
    cleaned = re.sub(r'<!--.*?-->', '', cleaned)

    # Remove <ref>...</ref> tags
    cleaned = re.sub(r'<ref[^>]*>.*?</ref>', '', cleaned)

    # Remove remaining <ref> or <name=...> tags
    cleaned = re.sub(r'<[^>]+>', '', cleaned)

    # Remove bold/italic markup ''' and '' (triple and double apostrophes)
    cleaned = re.sub(r"'''+|''", '', cleaned)

    # Remove section headers like === ''Harry Potter and the Goblet of Fire'' ===
    cleaned = re.sub(r'={2,}.+?={2,}', '', cleaned)

    # Remove infobox-style key-value pairs (e.g., spouse =, children =)
    cleaned = re.sub(r'\b\w+\s*=\s*[^|]+', '', cleaned)

    # Remove stray '|' characters
    cleaned = re.sub(r'\|', '', cleaned)

    # Remove extra spaces and newlines
    cleaned = ' '.join(cleaned.split())

    return cleaned




In [9]:
# extracting the keywords
def extract_keywords(sentence):
  words = re.findall(r'\w+', sentence.lower())
  significant_words = [word for word in words if word not in STOPWORDS and len(word) > 2]
  keyword_counts =  Counter(significant_words)
  keywords = [word for word, _ in keyword_counts.most_common(3)]
  return ', '.join(keywords) if keywords else 'N/A'

In [10]:
def split_text_into_sentences(wikitext):
    sentences = sent_tokenize(wikitext)
    return sentences

In [11]:
input_names = input("Enter the name: ").split(',')
input_names = [name.strip() for name in input_names]

wiki_data = []

for name in input_names:
  print(f'Fetching info for {name}')
  wikitext = fetch_wikiperdia_text(name)
  if wikitext:
    sentences = split_text_into_sentences(wikitext)
    for sentence in sentences:
      keywords = extract_keywords(sentence)
      cleaned_sentence = clean_sentence(sentence)
      wiki_data.append({
          'Name': name,
          'Sentence': cleaned_sentence,
          'Keywords': keywords
      })


Enter the name: Son Goku
Fetching info for Son Goku


en.wikipedia.org (parse) Goku
en.wikipedia.org (imageinfo) File:GokumangaToriyama.png
Goku (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Gokumanga...
  infobox: <dict(18)> name, franchise, image, caption, species, oc...
  iwlinks: <list(3)> https://commons.wikimedia.org/wiki/Category:G...
  pageid: 572605
  parsetree: <str(134521)> <root><template><title>Short descriptio...
  requests: <list(2)> parse, imageinfo
  title: Goku
  wikibase: Q2142
  wikidata_url: https://www.wikidata.org/wiki/Q2142
  wikitext: <str(101364)> {{Short description|Fictional character ...
}


In [12]:
wiki_data

[{'Name': 'Son Goku',
  'Sentence': '{{pp',
  'Keywords': 'dragon, ball, character'},
 {'Name': 'Son Goku',
  'Sentence': "Enough. Dragon Ball: Sleeping Princess in Devil's Castle and Dragon Ball: Mystical Adventure, DBZ ep.",
  'Keywords': 'dragon, ball, son'},
 {'Name': 'Son Goku',
  'Sentence': '128, one loop in ep.',
  'Keywords': '128, one, loop'},
 {'Name': 'Son Goku',
  'Sentence': '202}}) * Stephanie Nadolny (Funimation; young) * Colleen Clinkenbeard (Funimation; young) * Kristen McGuire (Funimation; infant) (Harmony Gold) * Nesty Ramirez (Creative Products Corp.) * Saffron Henderson (BLT{{refnDB eps.',
  'Keywords': 'refn, group, funimation'},
 {'Name': 'Son Goku',
  'Sentence': '1-13, Dragon Ball: Curse of the Blood Rubies}}) * Ian James Corlett (Ocean{{refnDBZ eps.',
  'Keywords': 'refn, group, dragon'},
 {'Name': 'Son Goku',
  'Sentence': '1-50 [edited into eps.',
  'Keywords': 'edited, eps'},
 {'Name': 'Son Goku',
  'Sentence': '1-37], Dragon Ball Z: The Tree of Might}}) *

In [13]:
df = pd.DataFrame(wiki_data)
df

,Name,Sentence,Keywords
0,Son Goku,{{pp,"dragon, ball, character"
1,Son Goku,Enough. Dragon Ball: Sleeping Princess in Devi...,"dragon, ball, son"
2,Son Goku,"128, one loop in ep.","128, one, loop"
3,Son Goku,202}}) * Stephanie Nadolny (Funimation; young)...,"refn, group, funimation"
4,Son Goku,"1-13, Dragon Ball: Curse of the Blood Rubies}}...","refn, group, dragon"
...,...,...,...
214,Son Goku,',"2022, website, collider"
215,Son Goku,While Anime Now's Richard Eisenbeis believes G...,"url, date, archive"
216,Son Goku,His rematch with Frieza was criticized for how...,"ref, dragon, ball"
217,Son Goku,His decision to let Cell have a senzu bean wit...,"dragon, ball, ref"


In [14]:
df.to_csv('Goku.csv', index=False)
print('Save to df')

Save to df


In [18]:
csv_file = '/content/Goku.csv' # Change the filename to match the saved file
df = pd.read_csv(csv_file)

txt_file = csv_file.replace('csv', 'txt')

with open(txt_file, 'w') as txt_f:
    for i, rows in df.iterrows():
        name = rows.get('Name', 'No Name Found')
        sentence = rows.get('Sentence', 'No Sentence Found')
        keywords = rows.get('Keywords', 'No Keywords Found')
        txt_f.write(f'Name: {name}\n')
        txt_f.write(f'Content: {sentence}\n')
        txt_f.write(f'Keywords: {keywords}\n')
        txt_f.write('\n')
